# Content-Based Filtering

# Data mining

In [1]:
import pandas as pd
df = pd.read_csv("imdb_top_1000.csv")

# Data cleaning

droping columns of data that aren't necessary for recom sys

In [2]:
df.drop(columns=['Poster_Link','Certificate','Overview'], inplace=True)

157 fields had missing data; about 15%. safe to delete.

In [3]:
print(df.isnull().sum())
#to-do: check if the dropped rows were important 
df.dropna(inplace=True)
print(df)

Series_Title       0
Released_Year      0
Runtime            0
Genre              0
IMDB_Rating        0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64
                 Series_Title Released_Year  Runtime  \
0    The Shawshank Redemption          1994  142 min   
1               The Godfather          1972  175 min   
2             The Dark Knight          2008  152 min   
3      The Godfather: Part II          1974  202 min   
4                12 Angry Men          1957   96 min   
..                        ...           ...      ...   
990              Giù la testa          1971  157 min   
991            Kelly's Heroes          1970  144 min   
992           The Jungle Book          1967   78 min   
994        A Hard Day's Night          1964   87 min   
997     From Here to Eternity          1953  118 min   

                            Genre  IMDB_R

turn released year into numeric data
and check the min and max value

In [4]:
# print(df.where(df["Released_Year"]=='PG').dropna())
df["Released_Year"]=pd.to_numeric(df["Released_Year"],errors="coerce")
df = df.dropna(subset=["Released_Year"])
print(df["Released_Year"].unique())
print(df["Released_Year"].min())
print(df["Released_Year"].max())

[1994. 1972. 2008. 1974. 1957. 2003. 1993. 2010. 1999. 2001. 1966. 2002.
 1990. 1980. 1975. 2019. 2014. 1998. 1997. 1995. 1991. 1977. 1954. 2011.
 2006. 2000. 1988. 1985. 1968. 1960. 1942. 1936. 1931. 2018. 2016. 2017.
 2012. 2009. 1981. 1979. 1964. 2004. 1992. 1987. 1986. 1984. 1983. 1976.
 1973. 1971. 1965. 1962. 1959. 1958. 1952. 1944. 1941. 1927. 2013. 2007.
 2005. 1989. 1963. 1955. 1950. 1948. 2015. 1996. 1982. 1978. 1967. 1951.
 1949. 1940. 1939. 1934. 1970. 1969. 1961. 1946. 1930. 1925. 1947. 1938.
 1933. 1935. 1953.]
1925.0
2019.0


## feature extraction

extracting feature "Decade" and assigning numerical values for each decade.

In [5]:
df["Decade"]= ((df["Released_Year"]-1920) // 10) + 1
df["Decade"]=df["Decade"].astype("int16")
print(df["Decade"])

0      8
1      6
2      9
3      6
4      4
      ..
990    6
991    6
992    5
994    5
997    4
Name: Decade, Length: 749, dtype: int16


turn Runtime column readable. 142 min is turned into 142.

In [6]:
df["Runtime"]=df["Runtime"].str.replace(' min','').astype("int16")
print(df["Runtime"])

0      142
1      175
2      152
3      202
4       96
      ... 
990    157
991    144
992     78
994     87
997    118
Name: Runtime, Length: 749, dtype: int16


using MultiLabelBinarizer to turn genres into numerical label values

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer
# split genres by comma 
df["Genre"] = df["Genre"].astype(str).str.split(", ")

In [8]:
df= df.dropna()
df = df.reset_index(drop=True)
print(df)

                 Series_Title  Released_Year  Runtime  \
0    The Shawshank Redemption         1994.0      142   
1               The Godfather         1972.0      175   
2             The Dark Knight         2008.0      152   
3      The Godfather: Part II         1974.0      202   
4                12 Angry Men         1957.0       96   
..                        ...            ...      ...   
744              Giù la testa         1971.0      157   
745            Kelly's Heroes         1970.0      144   
746           The Jungle Book         1967.0       78   
747        A Hard Day's Night         1964.0       87   
748     From Here to Eternity         1953.0      118   

                              Genre  IMDB_Rating  Meta_score  \
0                           [Drama]          9.3        80.0   
1                    [Crime, Drama]          9.2       100.0   
2            [Action, Crime, Drama]          9.0        84.0   
3                    [Crime, Drama]          9.0        90.

In [9]:
mlb = MultiLabelBinarizer()
# encode them into vectors of 0s and 1s
genre_encoded = mlb.fit_transform(df["Genre"])
print(len(genre_encoded))
# create a new data frame with the vectors and labels of genres
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
print(len(genre_df))
# concatinate the two DFs
df = pd.concat([df, genre_df], axis=1)
df= df.dropna()
df = df.reset_index(drop=True)
print(df)

749
749
                 Series_Title  Released_Year  Runtime  \
0    The Shawshank Redemption         1994.0      142   
1               The Godfather         1972.0      175   
2             The Dark Knight         2008.0      152   
3      The Godfather: Part II         1974.0      202   
4                12 Angry Men         1957.0       96   
..                        ...            ...      ...   
744              Giù la testa         1971.0      157   
745            Kelly's Heroes         1970.0      144   
746           The Jungle Book         1967.0       78   
747        A Hard Day's Night         1964.0       87   
748     From Here to Eternity         1953.0      118   

                              Genre  IMDB_Rating  Meta_score  \
0                           [Drama]          9.3        80.0   
1                    [Crime, Drama]          9.2       100.0   
2            [Action, Crime, Drama]          9.0        84.0   
3                    [Crime, Drama]          9.0   

setting dtype of IMDB_Rating and Meta_score

In [10]:
print(df["IMDB_Rating"].isna().sum())
print(df["Meta_score"].isna().sum())
df["IMDB_Rating"]= df["IMDB_Rating"].astype("float16")
df["Meta_score"]= df["Meta_score"].astype("int8")
print(df["IMDB_Rating"].head(10))
print(df["Meta_score"].head(10))

0
0
0    9.296875
1    9.203125
2    9.000000
3    9.000000
4    9.000000
5    8.898438
6    8.898438
7    8.898438
8    8.796875
9    8.796875
Name: IMDB_Rating, dtype: float16
0     80
1    100
2     84
3     90
4     96
5     94
6     94
7     94
8     74
9     66
Name: Meta_score, dtype: int8


C:\Users\sahar-rayane\AppData\Roaming\Python\Python312\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


checking the number of unique directors. every director appears ~1.7 on avg in the dataset. not a strong feature to compare fields.

In [11]:
print(df["Director"].nunique())

426


checking no of unique actors

In [12]:
# print(df[["Star1","Star2","Star3","Star4"]].nunique())
df["Stars"] = df[["Star1", "Star2", "Star3", "Star4"]].values.tolist()
# print(df["Stars"].head(10))
unique_stars = pd.unique(df[["Star1", "Star2", "Star3", "Star4"]].values.ravel())
print("Number of unique actors:", len(unique_stars))

Number of unique actors: 2030


no of unique stars and directors is too much that's about 2500 fields in a sparse matrix. building the system on Genres + Decade + Runtime + IMDB_Rating

scaling runtime and imdb rating

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numeric_matrix = scaler.fit_transform(df[['Runtime','IMDB_Rating']])
numeric_df = pd.DataFrame(numeric_matrix, columns=['Runtime_scaled','Rating_scaled'])

onehotencode decade

In [14]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output= False)
decade_matrix = ohe.fit_transform(df[['Decade']])
decade_df = pd.DataFrame(decade_matrix, columns=ohe.get_feature_names_out(['Decade']))

# final features matrix

In [15]:
from numpy import hstack
X = pd.concat([genre_df, decade_df, numeric_df], axis=1)

# similarity

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(X)
print(similarity_matrix)

[[1.         0.59880561 0.51365694 ... 0.00499497 0.01140651 0.31083551]
 [0.59880561 1.         0.67175193 ... 0.0063072  0.01440312 0.28126275]
 [0.51365694 0.67175193 1.         ... 0.00458387 0.01046772 0.2537999 ]
 ...
 [0.00499497 0.0063072  0.00458387 ... 1.         0.25066143 0.00291978]
 [0.01140651 0.01440312 0.01046772 ... 0.25066143 1.         0.0066676 ]
 [0.31083551 0.28126275 0.2537999  ... 0.00291978 0.0066676  1.        ]]


## step 1: Find index of user's input favorite

In [19]:
user_favorite = input("Enter title of your favorite movie: ")
row = df[df["Series_Title"].str.lower()==user_favorite.lower()]
if (row.empty):
    print("Your favorite movie was not found in the dataset.")
else:
    index= row.index[0]
    print(f"Movie '{user_favorite}' found at index {index}")

Movie 'Slumdog Millionaire' found at index 234


In [20]:
import numpy as np
sorted_similarity_matrix = np.argsort(-similarity_matrix, axis=1)
top_indices_movies= sorted_similarity_matrix[index][1:6]
top_titles= df.iloc[top_indices_movies]["Series_Title"].tolist()
print("You might also like:")
for title in top_titles:
    print("-", title)
# print(top_titles)

You might also like:
- Bom Yeoareum Gaeul Gyeoul Geurigo Bom
- The Notebook
- Pride & Prejudice
- Fa yeung nin wah
- Brokeback Mountain
